In [1]:
import cv2
import numpy as np
from scipy.ndimage import gaussian_filter, sobel

#### Scale Space Construction

In [2]:
def generate_octave(image, num_scales, sigma):
    octave = []
    for i in range(num_scales):
        image = gaussian_filter(image, ((2 ** 0.5) ** i) * sigma)
        octave.append(image)
    return np.array(octave)

In [3]:
def generate_gaussian_pyramid(image, num_octaves, num_scales, sigma):
    gaussian_pyramid = []
    scales = []

    for _ in range(num_octaves):
        octave_scales = []
        octave = generate_octave(image, num_scales, sigma)
        gaussian_pyramid.append(octave)
        # Append the initial sigma for the first image in the octave
        octave_scales.append(sigma)

        # Compute the sigma for subsequent images in the octave
        for _ in range(1, num_scales):
            sigma *= 2 ** (1 / num_scales)
            octave_scales.append(sigma)

        scales.append(octave_scales)

        image = cv2.resize(
            image, (image.shape[1] // 2, image.shape[0] // 2), interpolation=cv2.INTER_NEAREST)

    return gaussian_pyramid, scales

In [4]:
def generate_DoG_pyramid(gaussian_pyramid):
    DoG_pyramid = []
    for octave in gaussian_pyramid:
        DoG_octave = [octave[i + 1] - octave[i] for i in range(len(octave) - 1)]
        DoG_pyramid.append(DoG_octave)
    return DoG_pyramid

In [5]:
def scale_space_constuction(image, num_octaves, num_scales, sigma):
    gaussian_pyramid, scales = generate_gaussian_pyramid(image, num_octaves, num_scales, sigma)
    DoG_pyramid = generate_DoG_pyramid(gaussian_pyramid)
    return DoG_pyramid, scales

In [6]:
# def find_keypoints(DoG_pyramid):
#     keypoints = []
#     for octave_index, octave in enumerate(DoG_pyramid):
#         for scale_index in range(1, len(octave) - 1):
#             current_scale = octave[scale_index]
#             lower_scale = octave[scale_index - 1]
#             upper_scale = octave[scale_index + 1]
            

#             is_maxima = (current_scale > lower_scale) & (current_scale > upper_scale)
#             is_minima = (current_scale < lower_scale) & (current_scale < upper_scale)
            
#             # Find indices where the conditions are met
#             maxima_indices = np.argwhere(is_maxima)
#             print(f"Octave{octave_index}, Scale{scale_index}, No of max indicies:", len(maxima_indices))
#             minima_indices = np.argwhere(is_minima)
#             print(f"Octave{octave_index}, Scale{scale_index}, No of min indicies:", len(minima_indices))

#             keypoints.extend([(i, j, octave_index) for i, j in maxima_indices])
#             keypoints.extend([(i, j, octave_index) for i, j in minima_indices])
#         print("\n")

#     print("Total keypoints:", len(keypoints))
#     return keypoints


def find_keypoints(DoG_pyramid):
    keypoints = []
    for octave_index, octave in enumerate(DoG_pyramid):
        for scale_index in range(1, len(octave) - 1):
            current_scale = octave[scale_index]
            lower_scale = octave[scale_index - 1]
            upper_scale = octave[scale_index + 1]
            
            # Define the neighborhood indices
            neighborhood_indices = [
                (-1, -1), (-1, 0), (-1, 1),
                (0, -1), (0, 0), (0, 1),
                (1, -1), (1, 0), (1, 1)
            ]
            
            maxima_count = 0
            minima_count = 0
            for i in range(1, current_scale.shape[0] - 1):
                for j in range(1, current_scale.shape[1] - 1):
                    pixel = current_scale[i, j]
                    is_maxima = True
                    is_minima = True
                    for di, dj in neighborhood_indices:
                        if not (
                            lower_scale[i + di, j + dj] < pixel and upper_scale[i + di, j + dj] < pixel
                        ):
                            is_maxima = False
                        if not (
                            lower_scale[i + di, j + dj] > pixel and upper_scale[i + di, j + dj] > pixel
                        ):
                            is_minima = False
                    
                    if is_maxima:
                        maxima_count += 1
                        keypoints.append((i, j, octave_index))
                    elif is_minima:
                        minima_count += 1
                        keypoints.append((i, j, octave_index))
            
            print(f"Octave{octave_index}, Scale{scale_index}, No of maxima:", maxima_count)
            print(f"Octave{octave_index}, Scale{scale_index}, No of minima:", minima_count)
            print()
            
    print("Total keypoints:", len(keypoints))
    return keypoints


In [7]:
def refine_keypoints(keypoints, DoG_pyramid, contrast_threshold=0.04, edge_threshold=10):
    refined_keypoints = []

    for i, j, octave_index in keypoints:
        current_octave = DoG_pyramid[octave_index]
        current_img = current_octave[1]

        # Check if the keypoint is within the image boundaries
        if (i > 0 and i < current_img.shape[0] - 1 and
                j > 0 and j < current_img.shape[1] - 1):

            # Extract the current, upper, and lower scales
            current_scale = current_img[i, j]
            lower_scale = current_octave[0][i, j]
            upper_scale = current_octave[2][i, j]

            # Compute the difference of Gaussians
            delta_dog = np.abs(current_scale - lower_scale) + \
                np.abs(current_scale - upper_scale)

            # Check if the keypoint is a local extremum
            if (current_scale > lower_scale and current_scale > upper_scale and
                    current_scale > contrast_threshold * delta_dog):

                # Compute the curvature ratio
                Dxx = current_img[i, j + 1] + \
                    current_img[i, j - 1] - 2 * current_scale
                Dyy = current_img[i + 1, j] + \
                    current_img[i - 1, j] - 2 * current_scale
                Dxy = (current_img[i + 1, j + 1] - current_img[i + 1, j - 1] -
                       current_img[i - 1, j + 1] + current_img[i - 1, j - 1]) / 4
                trace = Dxx + Dyy
                determinant = Dxx * Dyy - Dxy ** 2
                curvature_ratio = trace ** 2 / \
                    determinant if determinant != 0 else float('inf')

                # Check if the curvature ratio satisfies the edge threshold
                if curvature_ratio < (edge_threshold + 1) ** 2 / edge_threshold:
                    refined_keypoints.append((i, j, octave_index))

    print("Total refined_keypoints:", len(refined_keypoints))
    return refined_keypoints

In [8]:
def compute_gradients(image):
    dx = sobel(image, axis=1)
    dy = sobel(image, axis=0)
    magnitude = np.sqrt(dx ** 2 + dy ** 2)
    orientation = np.arctan2(dy, dx)
    magnitude = magnitude.astype(np.float32)

    return magnitude, orientation

In [9]:
def find_peaks(hist):
    peaks = []
    for i in range(1, len(hist) - 1):
        if hist[i] > 0.8 * max(hist[i-1], hist[i], hist[i+1]):
            peaks.append(i)
    return peaks

In [10]:
def assign_orientation(refined_keypoints, DoG_pyramid):
    oriented_keypoints = []

    for i, j, octave_index in refined_keypoints:
        current_octave = DoG_pyramid[octave_index]
        current_img = current_octave[1]

        # Define window around the keypoint
        window_size = 30  # Adjust this according to your needs
        i_min = max(0, i - window_size)
        i_max = min(current_img.shape[0], i + window_size)
        j_min = max(0, j - window_size)
        j_max = min(current_img.shape[1], j + window_size)
        window = current_img[i_min:i_max, j_min:j_max]

        magnitude, orientation = compute_gradients(window)

        # Quantize orientation to 36 bins
        orientation_bins = ((orientation * (180 / np.pi) + 180) / 10).astype(int)

        # Create histogram using numpy's histogram function
        hist, _ = np.histogram(orientation_bins.flatten(), bins=range(37), weights=magnitude.flatten())
        
        # Assign keypoint orientation to the bin with max magnitude
        max_bin = np.argmax(hist)
        
        max_orientation = max_bin * 10 - 180  # Convert back to degrees
        
        oriented_keypoints.append((i, j, max_orientation))

    return oriented_keypoints

In [11]:
def calculate_orientation_histogram(magnitude, orientation, keypoint_x, keypoint_y, window_size=16, num_bins=8):
    # Initialize histogram
    histogram = np.zeros(num_bins)

    # Define window around keypoint
    half_window = window_size // 2

    # Loop through window and accumulate histogram
    for i in range(-half_window, half_window):
        for j in range(-half_window, half_window):
            x = keypoint_x + i
            y = keypoint_y + j

            # Ensure pixel is within image bounds
            if x >= 0 and x < magnitude.shape[0] and y >= 0 and y < magnitude.shape[1]:
                # Calculate gradient orientation relative to keypoint orientation
                relative_orientation = orientation[x, y] - \
                    orientation[keypoint_x, keypoint_y]

                # Map orientation to histogram bins
                bin_index = np.clip(
                    int((relative_orientation + 180) * num_bins / 360), 0, num_bins-1)

                # Accumulate magnitude into histogram bin
                histogram[bin_index] += magnitude[x, y]

    return histogram


def calculate_descriptor_vector(image, keypoints, window_size=16, num_bins=8):
    magnitude, orientation = compute_gradients(image)
    descriptor_vectors = []

    half_window = window_size // 2

    for keypoint in keypoints:
        keypoint_x, keypoint_y, _ = keypoint

        # Define window around keypoint
        x_min = max(0, keypoint_x - half_window)
        x_max = min(magnitude.shape[0], keypoint_x + half_window)
        y_min = max(0, keypoint_y - half_window)
        y_max = min(magnitude.shape[1], keypoint_y + half_window)

        window_magnitude = magnitude[x_min:x_max, y_min:y_max]
        window_orientation = orientation[x_min:x_max, y_min:y_max]

        # Calculate relative orientation and bin index
        relative_orientation = window_orientation - \
            orientation[keypoint_x, keypoint_y]
        bin_index = np.clip(np.floor(
            (relative_orientation + 180) / (360 / num_bins)), 0, num_bins - 1).astype(int)

        # Accumulate magnitudes into histogram bins
        histogram = np.zeros(num_bins)
        np.add.at(histogram, bin_index, window_magnitude)

        # Normalize histogram
        normalized_histogram = histogram / np.linalg.norm(histogram)
        descriptor_vectors.append(normalized_histogram)

    return descriptor_vectors

In [12]:
# def draw_keypoints(image, keypoints, orientations):
#     output_image = np.copy(image)

#     for kp, orientation in zip(keypoints, orientations):
#         x, y, _ = kp
#         scale = 1  # Scale factor for visualizing descriptors

#         # Draw the keypoint as a point
#         cv2.circle(output_image, (int(y), int(x)), 3, (0, 255, 0), -1)

#         # Compute endpoint of descriptor line
#         endpoint_x = x + scale * np.sin(orientation)
#         endpoint_y = y + scale * np.cos(orientation)
#         endpoint = (int(endpoint_y), int(endpoint_x))

#         # Draw the descriptor as a line
#         cv2.line(output_image, (int(y), int(x)), endpoint, (255, 0, 0), 1)

#     return output_image

---

In [13]:
# Read The Image
image = cv2.imread("collection 2/q3.jpg")

img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Define parameters
num_octaves = 4
num_scales = 5
sigma = 1.6
threshold = 0.03

In [14]:
# 1st step: Scale Space Constuction
DoG_pyramid, scales = scale_space_constuction(img, num_octaves, num_scales, sigma)

In [15]:
# 2nd step: Extrema Detection & Thresholding
keypoints = find_keypoints(DoG_pyramid)

Octave0, Scale1, No of maxima: 3832
Octave0, Scale1, No of minima: 8271

Octave0, Scale2, No of maxima: 7306
Octave0, Scale2, No of minima: 9711

Octave1, Scale1, No of maxima: 3150
Octave1, Scale1, No of minima: 3392

Octave1, Scale2, No of maxima: 3189
Octave1, Scale2, No of minima: 4944

Octave2, Scale1, No of maxima: 1994
Octave2, Scale1, No of minima: 2284

Octave2, Scale2, No of maxima: 1643
Octave2, Scale2, No of minima: 1117

Octave3, Scale1, No of maxima: 637
Octave3, Scale1, No of minima: 875

Octave3, Scale2, No of maxima: 912
Octave3, Scale2, No of minima: 1333

Total keypoints: 54590


In [16]:
print("No of keypoints:", len(keypoints))
keypoints[0]

No of keypoints: 54590


(1, 412, 0)

In [17]:
refined_keypoints = refine_keypoints(keypoints, DoG_pyramid, contrast_threshold=0.03, edge_threshold=10)

C:\Users\dell\AppData\Local\Temp\ipykernel_5372\3416151922.py:18: RuntimeWarning: overflow encountered in scalar add
  delta_dog = np.abs(current_scale - lower_scale) + \
C:\Users\dell\AppData\Local\Temp\ipykernel_5372\3416151922.py:26: RuntimeWarning: overflow encountered in scalar add
  Dxx = current_img[i, j + 1] + \
C:\Users\dell\AppData\Local\Temp\ipykernel_5372\3416151922.py:18: RuntimeWarning: overflow encountered in scalar subtract
  delta_dog = np.abs(current_scale - lower_scale) + \
C:\Users\dell\AppData\Local\Temp\ipykernel_5372\3416151922.py:19: RuntimeWarning: overflow encountered in scalar subtract
  np.abs(current_scale - upper_scale)
C:\Users\dell\AppData\Local\Temp\ipykernel_5372\3416151922.py:28: RuntimeWarning: overflow encountered in scalar add
  Dyy = current_img[i + 1, j] + \
C:\Users\dell\AppData\Local\Temp\ipykernel_5372\3416151922.py:30: RuntimeWarning: overflow encountered in scalar subtract
  Dxy = (current_img[i + 1, j + 1] - current_img[i + 1, j - 1] -
C:\U

Total refined_keypoints: 15453


In [18]:
# 3rd step: Orientation Assignment
orientations = assign_orientation(refined_keypoints, DoG_pyramid)

In [19]:
# 4th step: Keypoints Descriptors
descriptors = calculate_descriptor_vector(img, refined_keypoints)

In [20]:
print(orientations)

[(1, 412, 90), (19, 517, 0), (19, 518, 0), (22, 530, 0), (24, 407, 90), (24, 408, 90), (24, 445, 0), (25, 463, 0), (25, 464, 0), (25, 465, 0), (25, 466, 0), (26, 463, 0), (27, 437, 0), (27, 438, 0), (27, 439, 0), (28, 431, 0), (28, 441, 0), (28, 442, 0), (28, 443, 0), (30, 441, 0), (30, 442, 0), (31, 440, 0), (31, 441, 0), (31, 442, 0), (31, 471, 0), (31, 535, 0), (31, 536, 0), (31, 798, 90), (32, 471, 0), (32, 535, 0), (32, 536, 0), (32, 537, 0), (33, 419, 80), (33, 470, 0), (33, 471, 0), (33, 535, 0), (33, 536, 0), (33, 537, 0), (34, 470, 0), (35, 417, 80), (35, 418, 80), (35, 451, 80), (35, 452, 80), (35, 453, 80), (36, 470, 0), (38, 426, 80), (39, 517, 0), (39, 535, 0), (39, 536, 0), (39, 537, 0), (40, 465, 80), (40, 536, 0), (40, 537, 0), (41, 465, 80), (42, 419, 80), (42, 450, 80), (42, 451, 80), (43, 419, 80), (43, 449, 80), (44, 420, 80), (44, 449, 80), (44, 453, 80), (45, 420, 80), (45, 460, 80), (46, 420, 80), (46, 454, 80), (47, 420, 80), (47, 454, 80), (48, 420, 80), (49, 4

In [21]:
def draw_keypoints_with_orientation(image, keypoints, orientations):
    # Create a copy of the original image to draw keypoints on
    output_image = np.copy(image)

    for keypoint, orientation in zip(keypoints, orientations):
        i, j, _ = keypoint
        # Draw a circle for the keypoint
        cv2.circle(output_image, (j, i), radius=1, color=(255, 0, 0),
                   thickness=1)  # Assuming (i, j) represents (row, column)
        # Calculate the endpoint of the line based on orientation
        angle = orientation[2] * 180 / np.pi
        endpoint_x = int(j + 10 * np.cos(angle))
        endpoint_y = int(i + 10 * np.sin(angle))
        # Draw a line inside the circle representing orientation
        cv2.line(output_image, (j, i), (endpoint_x, endpoint_y),
                 color=(0, 255, 0), thickness=1)

    return output_image

In [22]:
def draw_keypoints(image, keypoints):
    # Create a copy of the original image to draw keypoints on
    output_image = np.copy(image)

    for keypoint in keypoints:
        i, j, _ = keypoint
        cv2.circle(output_image, (j, i), radius=1, color=(255, 0, 0),
                   thickness=-1)  # Assuming (i, j) represents (row, column)

    return output_image

In [23]:
output_img = draw_keypoints_with_orientation(image, refined_keypoints, orientations)
output_img0 = draw_keypoints(image, refined_keypoints)
cv2.imwrite("q3_collection_out.jpg", output_img0)

True

In [24]:
len(refined_keypoints)

15453